In [5]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [6]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index', "b1(Y)", "b2(Y)", "b3(Y)", "b4(Y)", "b5(Y)", "b6(Y)", "b7(Y)", "b8(Y)", "b9(Y)", "b10(Y)", "b11(Y)", "b12(Y)", "b13(Y)"])
print("Dataframe shape: ", df.shape)
df.head()
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/test.csv")
test = test.drop(columns=['index', "b1(Y)", "b2(Y)", "b3(Y)", "b4(Y)", "b5(Y)", "b6(Y)", "b7(Y)", "b8(Y)", "b9(Y)", "b10(Y)", "b11(Y)", "b12(Y)", "b13(Y)"])
print("Dataframe shape: ", test.shape)

Dataframe shape:  (1400, 14)
Dataframe shape:  (400, 14)


In [7]:
y = df['label']
X = df.drop(columns=['label'])
y = y.to_numpy()
X = X.to_numpy()    
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [8]:
DSCs = {}
time = {}
accuracys = {}
learning_rates = [0.0001, ]
batch_sizes = [2000, 3000, 4000]
min_dl = [1e-7, 1e-6, 1e-5,]
num_rules = [3, 5, 7,]
df_results = pd.DataFrame(columns=['lr', 'bs', 'mdl', 'nr', 'accuracy', 'f1_micro', 'f1_macro'])
results = []
for lr in learning_rates:
    for bs in batch_sizes:
        for mdl in min_dl:
            for nr in num_rules:
                DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=lr, batch_size=bs,
                                lossfn="MSE", num_workers=1, min_dloss=mdl, precompute_rules=True)
                print(f"Training DSC with lr={lr}, bs={bs}, mdl={mdl}, nr={nr}")
                losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=nr, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
                y_pred = DSC.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                f1_micro = f1_score(y_test, y_pred, average='micro')
                f1_macro = f1_score(y_test, y_pred, average='macro')
                print(f"Accuracy: {acc}, F1 micro: {f1_micro}, F1 macro: {f1_macro}")
                new_row = pd.DataFrame({
                    'lr': [lr], 'bs': [bs], 'mdl': [mdl], 'nr': [nr],
                    'accuracy': [acc], 'f1_micro': [f1_micro], 'f1_macro': [f1_macro]
                })
                df_results = pd.concat([df_results, new_row])
                results.append({
                    'lr': lr, 'bs': bs, 'mdl': mdl, 'nr': nr,
                    'accuracy': acc, 'f1_micro': f1_micro, 'f1_macro': f1_macro
                })
                

df_results.to_csv("results_neg.csv", index=False)

Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=3
Optimization started
Processing epoch	500	0.2230	
Training time: 1207.46s, epochs: 500

Least training loss reached: 0.223
Accuracy: 0.6725, F1 micro: 0.6725, F1 macro: 0.6714136112522027
Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=5
Optimization started
Processing epoch	1	1.0000	

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16312\3816186185.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row])


Processing epoch	500	0.2176	
Training time: 1169.19s, epochs: 500

Least training loss reached: 0.218
Accuracy: 0.66, F1 micro: 0.66, F1 macro: 0.6598639455782312
Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=7
Optimization started
Processing epoch	500	0.2273	
Training time: 1196.09s, epochs: 500

Least training loss reached: 0.227
Accuracy: 0.6275, F1 micro: 0.6275, F1 macro: 0.6255317010033992
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=3
Optimization started
Processing epoch	500	0.2206	
Training time: 1192.99s, epochs: 500

Least training loss reached: 0.221
Accuracy: 0.665, F1 micro: 0.665, F1 macro: 0.6622643411634237
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=5
Optimization started
Processing epoch	500	0.2323	
Training time: 1202.33s, epochs: 500

Least training loss reached: 0.232
Accuracy: 0.6425, F1 micro: 0.6425, F1 macro: 0.6383203394224507
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=7
Optimization started
Processing epoch	500	0.2308	
Training 

In [9]:
df_results = pd.read_csv('results_neg.csv')
df_results = df_results.sort_values(by='accuracy', ascending=False)
df_results.head(5)

,lr,bs,mdl,nr,accuracy,f1_micro,f1_macro
14,0.0001,3000,1.000000e-06,7,0.6750,0.6750,0.666461
0,0.0001,2000,1.000000e-07,3,0.6725,0.6725,0.671414
25,0.0001,4000,1.000000e-05,5,0.6725,0.6725,0.667915
23,0.0001,4000,1.000000e-06,7,0.6700,0.6700,0.669992
3,0.0001,2000,1.000000e-06,3,0.6650,0.6650,0.662264


In [10]:
# corremos los 5 mejores modelos y vemos las reglas que generan
dscs = []
for i in range(5):
    lr = df_results.iloc[i]['lr']
    bs = int(df_results.iloc[i]['bs'])
    mdl = df_results.iloc[i]['mdl']
    nr = int(df_results.iloc[i]['nr'])
    DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=lr, batch_size=bs,
                    lossfn="MSE", num_workers=1, min_dloss=mdl, precompute_rules=True)
    dscs.append(DSC)
    print(f"Training DSC with lr={lr}, bs={bs}, mdl={mdl}, nr={nr}")
    losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                single_rules_breaks=nr, add_mult_rules=False,
                    column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
    y_pred = DSC.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    print(f"Accuracy: {acc}, F1 micro: {f1_micro}, F1 macro: {f1_macro}")
    print("Rules:")
    DSC.print_most_important_rules(classes=["0",  "1"], threshold=0.19)

Training DSC with lr=0.0001, bs=3000, mdl=1e-06, nr=7
Optimization started
Processing epoch	500	0.2262	
Training time: 1161.77s, epochs: 500

Least training loss reached: 0.226
Accuracy: 0.655, F1 micro: 0.655, F1 macro: 0.6465073388150311
Rules:


Most important rules for class 0

	[0.243] R38: 0.084 < b5(N) < 0.130
			0: 0.243	1: 0.000	Unc: 0.757

	[0.239] R69: 0.193 < b1(N) < 0.282
			0: 0.239	1: 0.000	Unc: 0.761

	[0.235] R22: 0.435 < b6(N) < 0.600
			0: 0.235	1: 0.000	Unc: 0.765

	[0.234] R7: b2(N) > 0.380
			0: 0.234	1: 0.000	Unc: 0.766

	[0.215] R4: 0.096 < b2(N) < 0.175
			0: 0.215	1: 0.000	Unc: 0.785

	[0.203] R35: -0.013 < b5(N) < 0.018
			0: 0.203	1: 0.000	Unc: 0.797

	[0.195] R8: b7(N) < -0.110
			0: 0.190	1: 0.010	Unc: 0.800

	[0.193] R65: -0.172 < b1(N) < -0.053
			0: 0.187	1: 0.013	Unc: 0.800

	[0.193] R57: -0.059 < b4(N) < -0.031
			0: 0.187	1: 0.013	Unc: 0.800

	[0.193] R13: 0.163 < b7(N) < 0.229
			0: 0.188	1: 0.011	Unc: 0.801

	[0.193] R70: 0.282 < b1(N) < 0.400
			0

In [16]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=0)
rf.fit(X, y)
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"Accuracy: {acc}, F1 micro: {f1_micro}, F1 macro: {f1_macro}")

Accuracy: 0.68, F1 micro: 0.68, F1 macro: 0.6767676767676768
